In [83]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.optimize as opt
from matplotlib import pylab as plt

wine = pd.read_csv('full.csv', sep=',', header=0)
wine = wine.sample(frac=1)

source_data_1P = wine.P
source_data_D = wine.D

data_1P = np.array(source_data_1P, dtype=float)    #source numpy array 
data_D = np.array(source_data_D, dtype=float)

source_data_1Perr=[0.05]    #error list
source_data_Derr=[5]

data_1Perr = np.array(source_data_Terr)    #error numpy array
data_Derr = np.array(source_data_Nerr)

In [84]:
print wine

      D    P
0  3.60  188
4  0.58   29
3  0.74   39
1  1.90   94
2  1.50   65


In [85]:
data_lnH = np.log(data_H)
data_1devT = np.reciprocal(data_T)
data_lnHerr = np.reciprocal(data_H)*data_Herr[0]
data_1devTerr = np.reciprocal(data_1devT**2)*data_Terr[0]

NameError: name 'data_H' is not defined

$$Standard~deviation~y~is:~~ err_y{(w_0, w_1)}=\sqrt{\sum\limits_{i=1}^n [(y_i-w_0-{w_1}x_i)^2]} $$ 

In [86]:
def err_y(w):
    
    sum = 0.0
    i = 0
    n = data_D.size if data_D.size <= data_1P.size else data_1P.size
    
    while i <= n-1:
        
        sum = sum + ((data_D[i]-w[0]-w[1]*data_1P[i])**2) 
        
        i=i+1
    return sum

In [87]:
x0 = [0.0, 0.0]
bnds = [[-100000.0, 1000000.0], [-100000.0, 1000000.0]]
turp = opt.minimize(err_y, x0, method = 'L-BFGS-B', bounds=bnds)
print turp.x

n = data_D.size if data_D.size <= data_1P.size else data_1P.size

print 'randomize sigma y =', math.sqrt(err_y(turp.x))/(n*(n-1))

[ 0.0976699   0.01887145]
randomize sigma y = 0.0108088305247


In [88]:
def average(arr):
    av = 0.0
    iv = 0
    n = arr.size-1
    while iv<=n:
        av = av + (arr[iv])
        iv=iv+1
    av = av/n
    return av 

$$Standard~randomize~deviation~w_1~is:~~ \sigma_{w_1}{(w_0, w_1)}=\frac{1}{\sqrt{n}}\sqrt{{\frac{\langle{y^2}\rangle-\langle{y}\rangle^2}{\langle{x^{2}}\rangle-\langle{x}\rangle^2}-w_1^2}} $$ 
$$Standart~randomize~deviation~w_0~is:~~ \sigma_{w_0}{(w_0, w_1)}=\sigma_{w_1}{(w_0, w_1)}\sqrt{\langle{x^2}\rangle-\langle{x}\rangle^2}$$

In [91]:
def sigma_w(w): #randomize deviation
    
    sigma__w = [0, 0]
    n = data_D.size if data_D.size <= data_1P.size else data_1P.size
        
    sigma__w[1] = math.sqrt(math.fabs((((average(data_D**2)-average(data_D)**2)/
                             (average(data_1P**2)-(average(data_1P))**2))-w[1]**2)/n))
    sigma__w[0] = sigma__w[1]*math.sqrt(math.fabs(average(data_1P**2)-(average(data_1P))**2))
    
    return (sigma__w)

In [51]:
def sigma_wf(w): #randromize + statistical deviation
    
    sigma__wf = []
    j=0
    n = data_lnN.size if data_lnN.size <= data_T.size else data_T.size
    while j<=n-1:
        sigma__wf.append(0)
        j=j+1
        
    k = 0
    
    while k <= n-1:
        sigma__wf[k] = math.sqrt((data_lnNerr[0]/data_lnN[k])**2+(data_Terr[0]/data_T[k])**2)
        k = k + 1
    
    sigma__stat = average(np.array(sigma__wf))
    
    sigma__wf[1] = w[1]*sigma__stat
    
    sigma__w = [0, 0]
        
    sigma__w[1] = math.sqrt(math.fabs((((average(data_lnN**2)-average(data_lnN)**2)/
                             (average(data_T**2)-(average(data_T))**2))-w[1]**2)/n))
    sigma__w[1] = math.sqrt(sigma__wf[1]**2+sigma__w[1]**2)
    sigma__w[0] = sigma__w[1]*math.sqrt(math.fabs(average(data_T**2)-(average(data_T))**2))
    
    return (sigma__w)

In [92]:
print 'sigma rand = ', sigma_w(turp.x)
print 'sigma full = ', sigma_wf(turp.x)

sigma rand =  [0.13184096330550524, 0.003002839029319992]
sigma full =  [nan, nan]


$$Pearson's~chi-squared~test:~~ \chi{(w_0, w_1)}=\sum\limits_{i=2}^n \frac{[(y_i-w_0-{w_1}x_i)^2+{\sigma_y}^2+{w_1\sigma_x}^2]}{(w_0+{w_1}x_i)}$$ 

In [37]:
def chi(w): #turple #not work
    sum = 0.0
    i = 0
    n = data_lnN.size if data_lnN.size <= data_T.size else data_T.size
    while i <= n-1:
        if (data_T[i]*w[1]+w[0]) != 0:
            sum=sum+(((data_lnN[i]-(w[0]+w[1]*data_T[i]))**2)+((data_lnNerr[0])**2) +
                     ((w[1]*data_Terr[0])**2))/(w[0]+w[1]*data_T[i])
        i=i+1
    return sum

In [64]:
print 'chi = ', chi(turp.x)

chi =  -0.0637478871792
